# Police stations: Police_Stations_-_Map.csv
# Libraries: Libraries_-Locations_Hours_and_Contact_Information.csv
# Schools: Chicago Public Schools - School Loactions SY1011.geojson
# Todo: for each police report, find distance to nearest station, to nearest library, and to nearest school

In [319]:
import csv
import time
from math import sqrt
import geopandas as gpd
import pandas as pd
import shapely
from shapely.ops import nearest_points
import numpy as np
from scipy import ndimage
from scipy.spatial import cKDTree  

import matplotlib
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib.colors import ListedColormap
from matplotlib.ticker import MaxNLocator

In [ ]:
 def distance_to_nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
     """Find the nearest point and return the corresponding value from specified column."""
     distances = []
     # Find the geometry that is closest
     nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
     # Get the corresponding value from df2 (matching is based on the geometry)
     nearest_id = df2[nearest][src_column].get_values()[0]
     start_point = row['geometry']
     end_point_series = df2[nearest]['geometry']
     end_point = end_point_series.iloc[0]
     dist_to_point = start_point.distance(end_point)
     distances.append(dist_to_point)
     return nearest_id, dist_to_point

In [323]:
crimes = pd.read_pickle("crimes.pkl")
neighborhoods = pd.read_pickle("neighborhoods.pkl")
with_incomes = pd.read_pickle("with_incomes.pkl")

In [399]:
crimes2 = None
crimes2 = pd.read_pickle('crimes2.pkl')

In [400]:
crimes2.head()

,report_id,date,primary type,category,location description,arrest,community area,geometry
0,8265244,09/15/2011,BURGLARY,property,RESIDENCE,true,false,POINT (-87.771918163 41.95396528)
1,4991465,01/01/2001,DECEPTIVE PRACTICE,other,"SCHOOL, PRIVATE, GROUNDS",true,false,POINT (-87.660604177 42.001678618)
2,5147738,08/01/2001,OFFENSE INVOLVING CHILDREN,person,RESIDENCE,true,false,POINT (-87.66343916300001 41.802219519)
3,5281401,02/26/2001,OTHER OFFENSE,other,HOSPITAL BUILDING/GROUNDS,false,false,POINT (-87.61391851099999 41.802238398)
4,5622147,12/01/2001,OFFENSE INVOLVING CHILDREN,person,RESIDENCE,false,true,POINT (-87.719768573 41.931044901)


In [361]:
crimes = crimes.astype({'id': str})
type(crimes.iloc[0]['id'])

str

In [341]:
# Read in the school locations data, fix the type of the UNIT_ID column, create shapely points from X/Y
# and create the geopandas DataFrame
schools = pd.read_csv("school-locations-2010-2011.csv", index_col=2)
schools['UNIT_ID'] = schools.index
schools = schools.astype({'UNIT_ID': int})
schools['geometry'] = list(zip(schools['X'], schools['Y']))
schools['geometry'] = schools['geometry'].apply(shapely.geometry.Point)
schools = gpd.GeoDataFrame(schools)
schools.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 688 entries, 4610 to 4210
Data columns (total 10 columns):
geometry     688 non-null object
SCHOOL_NM    688 non-null object
SCHOOL_ID    688 non-null int64
SCH_ADDR     688 non-null object
GRADE_CAT    688 non-null object
GRADES       688 non-null object
SCH_TYPE     688 non-null object
X            688 non-null float64
Y            688 non-null float64
UNIT_ID      688 non-null int64
dtypes: float64(2), int64(2), object(6)
memory usage: 59.1+ KB


In [229]:
# Remove columns we don't plan to use
del schools['geometry']
del schools['SCHOOL_ID']
del schools['GRADES']

In [343]:
# Create unary_union object from the schools dataset
schools_unary_union = schools.unary_union

In [418]:
#print(schools['geometry'] == nearest_points(crimes.iloc[2]['geometry'], schools_unary_union))
print(crimes.iloc[2]['geometry'].distance(schools.iloc[100]['geometry']))
#print(crimes.iloc[2]['geometry'])

0.09386390354079342


In [407]:
# Locate the nearest school to each reported crime's location
start_time = time.time
unpackdf = pd.DataFrame(crimes2.apply(distance_to_nearest,
                                     geom_union=schools_unary_union,
                                     df1=crimes2,
                                     df2=schools,
                                     src_column='UNIT_ID',
                                     axis=1).tolist(),
                        columns = ['nearest_school_id',
                                   'nearest_school_distance'],
                        index=crimes2.index)
crimes2=pd.concat([unpackdf, crimes2], axis=1)
end_time = time.time
print("That took {0} seconds".format(end_time - start_time))

TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'builtin_function_or_method'

In [408]:
crimes2.head()

,nearest_school_id,nearest_school_distance,report_id,date,primary type,category,location description,arrest,community area,geometry
0,6190,[],8265244,09/15/2011,BURGLARY,property,RESIDENCE,true,false,POINT (-87.771918163 41.95396528)
1,4300,[],4991465,01/01/2001,DECEPTIVE PRACTICE,other,"SCHOOL, PRIVATE, GROUNDS",true,false,POINT (-87.660604177 42.001678618)
2,1110,[],5147738,08/01/2001,OFFENSE INVOLVING CHILDREN,person,RESIDENCE,true,false,POINT (-87.66343916300001 41.802219519)
3,1600,[],5281401,02/26/2001,OTHER OFFENSE,other,HOSPITAL BUILDING/GROUNDS,false,false,POINT (-87.61391851099999 41.802238398)
4,4850,[],5622147,12/01/2001,OFFENSE INVOLVING CHILDREN,person,RESIDENCE,false,true,POINT (-87.719768573 41.931044901)


In [348]:
# Locate the nearest school to each reported crime's location
start_time = time.time
unpackdf = pd.DataFrame(crimes.apply(distance_to_nearest,
                                     geom_union=schools_unary_union,
                                     df1=crimes,
                                     df2=schools,
                                     src_column='UNIT_ID',
                                     axis=1).tolist(),
                        columns = ['nearest_school_id',
                                   'nearest_school_distance'],
                        index=test_start.index)
crimes=pd.concat([unpackdf, crimes], axis=1)
end_time = time.time
print("That took {0} seconds".format(end_time - start_time))

IDs for origin 8265244, and destination 6190
IDs for origin 4991465, and destination 4300
IDs for origin 5147738, and destination 1110
IDs for origin 5281401, and destination 1600
IDs for origin 5622147, and destination 4850
IDs for origin 11020229, and destination 2020
IDs for origin 5746138, and destination 6750
IDs for origin 5868895, and destination 3341
IDs for origin 5969635, and destination 6040
IDs for origin 8187948, and destination 4890
IDs for origin 8263618, and destination 5220


AttributeError: ("'GeoSeries' object has no attribute '_geom'", 'occurred at index 8263618')

In [153]:
test_start['nearest_id'] = test_start.apply(nearest_id, geom_union=unary_union, df1=test_start, df2=test_end, src_column='id', axis=1)
test_start

Nearest IDs: ['epoint4']
Distances: [<shapely.geometry.point.Point object at 0xb2371d7f0>]
Nearest IDs: ['epoint5']
Distances: [<shapely.geometry.point.Point object at 0xb2371d828>]
Nearest IDs: ['epoint4']
Distances: [<shapely.geometry.point.Point object at 0xb2371d7f0>]


,id,geometry,nearest_id
spoint1,spoint1,POINT (1 0),epoint4
spoint2,spoint2,POINT (5 0),epoint5
spoint3,spoint3,POINT (3 5),epoint4


In [142]:
test_start['nearest_id'] = test_start.apply(nearest_id, geom_union=unary_union, df1=test_start, df2=test_end, src_column='id', axis=1)
distances = []
distance_indecies = []
for row in test_start.itertuples():
    start_point = getattr(row, 'geometry')
    end_point = test_end.loc[getattr(row, 'nearest_id')]['geometry']
    distances.append(start_point.distance(end_point))
    distance_indecies.append(row.id)
new_column = pd.Series(distances, index=distance_indecies)
test_start['nearest_distance'] = new_column
print(new_column)
test_start

spoint1    4.123106
spoint2    3.000000
spoint3    1.414214
dtype: float64


,id,geometry,nearest_id,nearest_distance
spoint1,spoint1,POINT (1 0),epoint4,4.123106
spoint2,spoint2,POINT (5 0),epoint5,3.000000
spoint3,spoint3,POINT (3 5),epoint4,1.414214


In [32]:
(test_start['nearest_id'], test_start['nearest_point']) = test_start.apply(nearest,
                                                                         geom_union=unary_union,
                                                                         df1=test_start,
                                                                         df2=test_end,
                                                                         src_column='id', axis=1)

Nearest(<class 'geopandas.geoseries.GeoSeries'>): epoint1    False
epoint2    False
epoint3    False
epoint4     True
epoint5    False
Name: geometry, dtype: bool
POINT (2 4)
Nearest(<class 'geopandas.geoseries.GeoSeries'>): epoint1    False
epoint2    False
epoint3    False
epoint4    False
epoint5     True
Name: geometry, dtype: bool
POINT (8 0)
Nearest(<class 'geopandas.geoseries.GeoSeries'>): epoint1    False
epoint2    False
epoint3    False
epoint4     True
epoint5    False
Name: geometry, dtype: bool
POINT (2 4)


ValueError: too many values to unpack (expected 2)

In [146]:
df = pd.DataFrame([[4, 9],] * 3, columns=['A', 'B'])
df
df['foo'], df['bar'] = df.apply(lambda x: pd.Series([1, 2], index=['foo', 'bar']), axis=1)
df

,A,B,foo,bar
0,4,9,foo,bar
1,4,9,foo,bar
2,4,9,foo,bar


In [151]:
start_uu = test_start.unary_union

I'm working on a personal-learning project where I'm trying to figure out what the nearest point from dataframe B is to each point in in dataframe A. I've got that bit working, but it only tells me what the nearest point is. I also want to know how far away the nearest point is, but I'm not figuring it out.

I can find the distance manually one point at a time, but my efforts to do so for the whole list have failed thus far.

Simplified examples of the two dataframes are:

    test_start = gpd.GeoDataFrame([{"id": "spoint1", "geometry": shapely.geometry.Point(1, 0)},
                                   {"id": "spoint2", "geometry": shapely.geometry.Point(5, 0)}])

    test_end = gpd.GeoDataFrame([{"id": "epoint1", "geometry": shapely.geometry.Point(-3, 3)},
                                 {"id": "epoint2", "geometry": shapely.geometry.Point(-1, 4)},
                                 {"id": "epoint3", "geometry": shapely.geometry.Point(5, 4)},
                                 {"id": "epoint4", "geometry": shapely.geometry.Point(2, 4)},
                                 {"id": "epoint5", "geometry": shapely.geometry.Point(8, 0)}
                                ])

The nearest_point method I've cargo-culted from [this tutorial](https://automating-gis-processes.github.io/2017/lessons/L3/nearest-neighbour.html) is:

    def nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
        """Find the nearest point and return the corresponding value from specified column."""
        # Find the geometry that is closest
        nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
        #nearest = df2[geom2_col] == nearest_points(df1, df2)[1]
        # Get the corresponding value from df2 (matching is based on the geometry)
        value = df2[nearest][src_column].get_values()[0]
        return value

which is called from:

    test_start['nearest_id'] = test_start.apply(nearest,
                                                geom_union=unary_union,
                                                df1=test_start,
                                                df2=test_end,
                                                src_column='id', axis=1)

What I want is another column in test_start that contains the distance between the start and end points. As mentioned above, I can do this manually with something like:

    print(test_start["geometry"][0].distance(test_end["geometry"][3]))

but I can't figure out how to do it for the whole test_start dataframe.

This is a very simplified data set. The real starting point data set has over 7 million rows and the end set has around 800, so efficiency may become a factor if the brute-force method takes half a day or more.

Using apply() seemed like it would be a path to get to what I wanted, but I can't figure out how to write it so the system knows which points are being compared.

Somewhat closer. This code gets past the indexing, etc. problems, but appears to select just one end point as being nearest to all of the start points.

     def distance_to_nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
         """Find the nearest point and return the corresponding value from specified column."""
         distances = []
         # Find the geometry that is closest
         nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
         # Get the corresponding value from df2 (matching is based on the geometry)
         nearest_id = df2[nearest][src_column].get_values()[0]
         start_point = row['geometry']
         end_point_series = df2[nearest]['geometry']
         end_point = end_point_series.iloc[0]
         dist_to_point = start_point.distance(end_point)
         distances.append(dist_to_point)
         return nearest_id, dist_to_point